In [4]:
import requests
import xmltodict
import json
import folium
import pandas as pd 

# 공공데이터포털 사이트로 크롤링 요청 위한 인증키와 요청 메세지
key = "y3hJyZd4gVr4xFzmKzqb53nioC68jdCSKnf8lZYuhvp9VOgSJ5N1GgtPP7fEVhcasTygTCSvigiqKj2syoMEyg%3D%3D"
url_cycle="http://openapi.jejusi.go.kr/rest/PublicBikeInfoService/getPublicBikeInfoList?ServiceKey={}&pageNo=1&numOfRows=10".format(key)
url_parking = "http://openapi.jejuits.go.kr/OPEN_API/pisInfo/getPisInfo?serviceKey={}&pageNo=1&numOfRows=100".format(key)

# 웹에 요청
content_cycle = requests.get(url_cycle).content

# xml형식은 정보 분리하기 힘들어서 json으로 파싱
dict_cycle = xmltodict.parse(content_cycle)

#제주도맵 folium에 불러오기
jeju_map = folium.Map(location=[33.389398,126.541236], tiles='stamen Terrain',zoom_start=12)

# json으로 변화된 정보를 구분해서 출력-공공자전거
jsonString_cycle = json.dumps(dict_cycle['rfcOpenApi']['body'], ensure_ascii = False)
jsonObj_cycle = json.loads(jsonString_cycle)
print('\n ----------------------------------데이터프레임으로 변환------------------------------')
df_cycle = pd.DataFrame(jsonObj_cycle['data']['list'])

# 지도에 위치를 마킹하기 위해 x,y좌표를 수동으로 입력해줌
df_c1 = pd.DataFrame({'x':[33.4845,33.4889,33.4908,33.4794,33.4852,33.4753,33.5112,33.5004,33.4998]}) # x좌표 lat
df_c2 = pd.DataFrame({'y':[126.5007,126.4906,126.4866,126.4772,126.4882,126.5154,126.5432,126.5297,126.5178]}) # y좌표 lon

#좌표를 리스트에 추가함
df_cycle = pd.concat([df_cycle,df_c1],axis=1)
df_cycle = pd.concat([df_cycle,df_c2],axis=1)
print(df_cycle)# 최종 리스트 출력

# for문을 반복하여 자전거대여소의 위치를 지도에 마킹 + 설명과 잔여 대여 대수를 확인할수있음.
for x , y , loc, num in zip(df_cycle.x , df_cycle.y , df_cycle.location ,df_cycle.retal_enable_num):
    if int(num) == 0:
        folium.CircleMarker([x,y],
                         radius=4,
                         color='red',
                         fill=True,
                         fill_color='red',
                         fill_opacity=0.7,
                         popup=loc+num
                       ).add_to(jeju_map)  # 대여가능 대수가 0대일시 적색등.
        
    elif int(num) < 5:
          folium.CircleMarker([x,y],
                         radius=4,
                         color='orange',
                         fill=True,
                         fill_color='orange',
                         fill_opacity=0.7,
                         popup=loc+num
                       ).add_to(jeju_map) #  5대 미만은 주황등.
            
    else :
        folium.CircleMarker([x,y],
                         radius=4,
                         color='green',
                         fill=True,
                         fill_color='green',
                         fill_opacity=0.7,
                         popup=loc+num
                       ).add_to(jeju_map) # 5대 이상은 초록등.

           
#HTML 파일로 저장    
jeju_map.save('./jeju_cycle.html')      


 ----------------------------------데이터프레임으로 변환------------------------------
         address           location retal_enable_num           title        x  \
0      연동 324-10     설문대여성문화센터 주차장내                2    01.설문대여성문화센터  33.4845   
1      연동 272-56      로얄쇼핑센터 앞 주차장내                0       02.로얄쇼핑센터  33.4889   
2      연동 2322-7      롯데시티호텔 뒤 주차장내                4       03.롯데시티호텔  33.4908   
3     노형동 3812-2         탐라도서관 주차장내                3        04.탐라도서관  33.4794   
4        연동 1499  국기로 대림아파트 동측 주차장내                0     05.국기로(대림A)  33.4852   
5     오라2동 898-8          제주아트센터 입구                1  06.아트센터(한라도서관)  33.4753   
6     일도2동 488-2         제주영락교회 맞은편                2         07.영락교회  33.5112   
7  이도2동 1176-124           제주벤처마루 앞                8         08.벤처마루  33.5004   
8     삼도1동 560-2     오라지구대 옆 공영주차장내                7        09.오라지구대  33.4998   

          y  
0  126.5007  
1  126.4906  
2  126.4866  
3  126.4772  
4  126.4882  
5  126.5154  
6  126.5432  